# Spectroscopic Data Reduction Part 3: Extracting the final wavelength-calibrated spectrum

This tutorial assumes you have gone through the [Trace](1-SpectroscopicTraceTutorial.ipynb) and [Wavelength Calibration](2-WavelengthCalibration) tutorials and have their results available.

## Authors
Adam Ginsburg, Kelle Cruz, Lia Corrales, Jonathan Sick, Adrian Price-Whelan, and Sam Grunblatt

## Learning Goals
* Extract a target 1D spectrum from a two-dimensional spectrum using an existing trace
* Apply a fitted wavelength solution to the data
* Fit a line profile to the wavelength-calibrated spectrum

## Keywords
Spectroscopy

## Summary
This tutorial, the third in a series, shows how to apply a trace and a wavelength solution to science data.  It then shows how to do basic analysis, i.e., line fitting.

In [ ]:
import requests

url = 'https://raw.githubusercontent.com/skgrunblatt/astropy-tutorials/main/tutorials/SpectroscopicDataReductionBasics/requirements.txt'
response = requests.get(url)

if response.status_code == 200:
    print(f"Required packages for this notebook:\n{response.text}")
else:
    print("Failed to retrieve the file.")

## Extract the science spectrum

First, we repeat the trace-and-extract process derived in [Part 1](1-SpectroscopicTraceTutorial.ipynb), but now for Deneb:

In [ ]:
from PIL import Image as PILImage
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [ ]:
from astropy.modeling.models import Linear1D
from astropy import constants
from astropy import units as u
from astropy.visualization import quantity_support
quantity_support()

In [ ]:
from io import BytesIO
import requests
import numpy as np

# example for a FITS image (uncomment this for final part of lab):
from astropy.io import fits
url = "https://raw.githubusercontent.com/skgrunblatt/astropy-tutorials/main/tutorials/SpectroscopicDataReductionBasics/betel6.fits"

# Fetch the image data from the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Create a file-like object from the response content
    image_array_2 = fits.open(BytesIO(response.content))[0].data
    print("Image loaded successfully!")
else:
    print("Failed to retrieve the image.")
print(image_array_2.shape)

In [ ]:
from astropy.modeling.polynomial import Polynomial1D
from astropy.modeling.models import Gaussian1D
from astropy.modeling.fitting import LevMarLSQFitter, LinearLSQFitter
linfitter = LinearLSQFitter()

In [ ]:
print(xvals.shape)
print(yaxis2.shape)

In [ ]:
yaxis2 = np.repeat(np.arange(40, 80)[:,None], image_array_2.shape[1], axis=1)
xvals = np.arange(image_array_2.shape[1])
weighted_yaxis_values2 = np.average(yaxis2, axis=0, weights=image_array_2[40:80,:] - np.median(image_array_2))
polymodel2 = Polynomial1D(degree=3)
fitted_polymodel2 = linfitter(polymodel2, xvals, weighted_yaxis_values2)
trace_center2 = fitted_polymodel2(xvals)

In [ ]:
npixels_to_cut = 15
trace_center = fitted_polymodel2(xvals)
cutouts = np.array([image_array_2[int(yval)-npixels_to_cut:int(yval)+npixels_to_cut, ii]
                    for yval, ii in zip(trace_center, xvals)])
cutouts.shape
mean_trace_profile = cutouts.mean(axis=0)

In [ ]:
spectrum2 = np.array([np.average(image_array_2[int(yval)-npixels_to_cut:int(yval)+npixels_to_cut, ii],
                                weights=mean_trace_profile)
                     for yval, ii in zip(trace_center2, xvals)])

Next, we retrieve the wavelength solution derived in Part 2.

In [ ]:
from astropy import units as u
from astropy.modeling.polynomial import Polynomial1D
from astropy.modeling.models import Gaussian1D, Linear1D
from astropy.modeling.fitting import LinearLSQFitter
from IPython.display import Image
# astroquery provides an interface to the NIST atomic line database
# from astroquery.nist import Nist

wlmodel = Linear1D(slope=-0.45128449, intercept=701.91734074)#Linear1D(slope=-0.10213643, intercept=562.3862495)
# Linear1D(slope=-0.45128449, intercept=701.91734074)>

In [ ]:
wavelengths = wlmodel(xvals) * u.nm

In [ ]:
import matplotlib.pyplot as plt
plt.plot(wavelengths, spectrum2)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Flux (arbitary units)')